## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-28-01-15-23 +0000,nyt,"Palestinian Leader Lays Out Succession Plan, E...",https://www.nytimes.com/2025/10/26/world/middl...
1,2025-10-28-01-11-46 +0000,nyt,Live Updates: Hurricane Melissa Turns Toward J...,https://www.nytimes.com/live/2025/10/21/weathe...
2,2025-10-28-01-03-33 +0000,nypost,Brooklyn Republican sends mailer tying Democra...,https://nypost.com/2025/10/27/us-news/brooklyn...
3,2025-10-28-01-00-00 +0000,wsj,China Pushes to Silence Victims of African Min...,https://www.wsj.com/world/asia/china-africa-mi...
4,2025-10-28-01-00-00 +0000,wsj,Argentina’s Slums Abandoned Peronism—And Hande...,https://www.wsj.com/world/americas/argentina-e...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
89,trump,49
31,china,19
337,new,18
254,he,14
43,milei,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
254,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...,143
292,2025-10-27-04-17-00 +0000,wsj,"Asian Markets Rise as U.S.-China Talks, Trump ...",https://www.wsj.com/finance/stocks/asian-marke...,108
31,2025-10-27-23-45-36 +0000,bbc,Trump does not rule out seeking third term - b...,https://www.bbc.com/news/articles/c797q57ple9o...,101
293,2025-10-27-04-09-01 +0000,nypost,"US, China talks sketch out rare earths, tariff...",https://nypost.com/2025/10/27/world-news/us-ch...,98
227,2025-10-27-12-47-34 +0000,cbc,When will Trump's tariff increase hit Canada? ...,https://www.cbc.ca/news/world/trump-tariff-can...,95


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
254,143,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...
262,78,2025-10-27-09-01-23 +0000,wsj,"Stock Market News, Oct. 27, 2025: Dow Hits Ne...",https://www.wsj.com/livecoverage/stock-market-...
28,54,2025-10-27-23-52-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...
68,53,2025-10-27-21-54-59 +0000,nypost,Storm chaser captures stunning eye of Hurrican...,https://nypost.com/2025/10/27/world-news/hurri...
170,47,2025-10-27-16-33-34 +0000,latimes,'CBS Evening News' co-anchor John Dickerson wi...,https://www.latimes.com/entertainment-arts/bus...
185,43,2025-10-27-15-42-00 +0000,nypost,Treasury Secretary Bessent says he felt ‘pain’...,https://nypost.com/2025/10/27/business/treasur...
143,43,2025-10-27-18-01-17 +0000,nypost,Andrew Cuomo narrows gap against Zohran Mamdan...,https://nypost.com/2025/10/27/us-news/andrew-c...
282,39,2025-10-27-06-46-40 +0000,nypost,Trump congratulates Argentina’s Javier Milei a...,https://nypost.com/2025/10/27/world-news/argen...
172,34,2025-10-27-16-28-00 +0000,wsj,A shortlist of candidates to be the next Fed c...,https://www.wsj.com/economy/central-banking/tr...
6,31,2025-10-28-01-00-00 +0000,wsj,Watering down dish soap. Trading down to groce...,https://www.wsj.com/economy/consumers/the-leng...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
